<a href="https://colab.research.google.com/github/AidanOD-tech/Legal-Tech-Uni-Assessment/blob/main/Assignment_Code_05_12_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
from datetime import datetime
import gspread
from google.colab import auth
from google.auth import default
from gspread.exceptions import SpreadsheetNotFound as SheetNotFound

# Authenticate and authorise Google Sheets. You will need accept Collab access to your google account.
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds) # I tried to use the UK spelling but then it runs into errors.

# Define candidate mapping
candidates = ["Vaughn Leland", "Jeremy Wendell", "Marilee Stone", "Thornton McLeash"]
name_to_id = {name: idx + 1 for idx, name in enumerate(candidates)} # # Create a mapping from candidate names to unique numeric IDs starting from 1 # This mapping is used to encode candidate names into numbers for encryption and ranking
id_to_name = {v: k for k, v in name_to_id.items()} # Create a reverse mapping from numeric IDs back to candidate names. # This is used to decode numeric data back into readable candidate names after decryption


# Updated professor preferences.
professor_preferences = {
    "Prof. Ashley": ["Thornton McLeash", "Vaughn Leland", "Jeremy Wendell", "Marilee Stone"],
    "Prof. Baker": ["Jeremy Wendell", "Marilee Stone", "Vaughn Leland", "Thornton McLeash"],
    "Prof. Barnes": ["Marilee Stone", "Jeremy Wendell", "Vaughn Leland", "Thornton McLeash"],
    "Prof. Caroline": ["Marilee Stone", "Vaughn Leland", "Jeremy Wendell", "Thornton McLeash"],
    "Prof. Culver": ["Jeremy Wendell", "Marilee Stone", "Vaughn Leland", "Thornton McLeash"],
    "Prof. Driscoll": ["Marilee Stone", "Thornton McLeash", "Vaughn Leland", "Jeremy Wendell"],
    "Prof. Eugene": ["Vaughn Leland", "Thornton McLeash", "Jeremy Wendell", "Marilee Stone"],
    "Prof. Ewing": ["Jeremy Wendell", "Marilee Stone", "Thornton McLeash", "Vaughn Leland"],
    "Prof. Farlow": ["Jeremy Wendell", "Marilee Stone", "Thornton McLeash", "Vaughn Leland"],
    "Prof. Graison": ["Marilee Stone", "Thornton McLeash", "Vaughn Leland", "Jeremy Wendell"],
    "Prof. Hampton": ["Jeremy Wendell", "Vaughn Leland", "Thornton McLeash", "Marilee Stone"],
    "Prof. Krebbs": ["Jeremy Wendell", "Vaughn Leland", "Thornton McLeash", "Marilee Stone"],
    "Prof. Lee": ["Vaughn Leland", "Jeremy Wendell", "Thornton McLeash", "Marilee Stone"],
    "Prof. Luce": ["Vaughn Leland", "Marilee Stone", "Jeremy Wendell", "Thornton McLeash"],
    "Prof. McFarland": ["Jeremy Wendell", "Marilee Stone", "Thornton McLeash", "Vaughn Leland"],
    "Prof. Packer": ["Jeremy Wendell", "Vaughn Leland", "Marilee Stone", "Thornton McLeash"],
    "Prof. Randolph": ["Jeremy Wendell", "Thornton McLeash", "Marilee Stone", "Vaughn Leland"],
    "Prof. Smithfield": ["Jeremy Wendell", "Thornton McLeash", "Marilee Stone", "Vaughn Leland"],
    "Prof. Wade": ["Jeremy Wendell", "Vaughn Leland", "Thornton McLeash", "Marilee Stone"],
    "Prof. Washburn": ["Jeremy Wendell", "Marilee Stone", "Vaughn Leland", "Thornton McLeash"],
    "Prof. Wentworth": ["Thornton McLeash", "Jeremy Wendell", "Vaughn Leland", "Marilee Stone"]
}

# RSA key generation
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num ** 0.5) + 1):
        if num % i == 0:
            return False
    return True

def find_nearest_prime(num):
    while not is_prime(num):
        num += 1
    return num

entropy1 = random.randint(50, 100)
entropy2 = random.randint(50, 100)
prime1 = find_nearest_prime(entropy1)
prime2 = find_nearest_prime(entropy2)
n = prime1 * prime2
phi_n = (prime1 - 1) * (prime2 - 1)

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def find_e(phi):
    for i in range(2, phi):
        if gcd(i, phi) == 1:
            return i

def find_d(e, phi):
    d = 1
    while (e * d) % phi != 1:
        d += 1
    return d

e = find_e(phi_n)
d = find_d(e, phi_n)

# Encryption and decryption
def encrypt(preference):
    return pow(preference, e, n)

def decrypt(encrypted_preference):
    return pow(encrypted_preference, d, n)

# Convert preferences to numeric and back
def encode_preferences(preferences):
    return int("".join(str(name_to_id[name]) for name in preferences))

def decode_preferences(encoded_preferences):
    try:
        return [id_to_name[int(d)] for d in str(encoded_preferences) if int(d) in id_to_name]
    except KeyError as e:
        print(f"Decoding error: Invalid ID {e} in {encoded_preferences}")
        return []

# Google Sheet helper functions
def create_or_open_sheet(encrypted_votes):
    try:
        return gc.open(encrypted_votes)
    except SheetNotFound:
        sheet = gc.create(encrypted_votes)
        sheet.share(None, perm_type='anyone', role='writer')
        worksheet = sheet.sheet1
        worksheet.append_row(["Professor", "Encrypted Vote", "Timestamp"])
        return sheet

def add_vote_to_sheet(encrypted_votes, professor, encrypted_vote):
    sheet = create_or_open_sheet(encrypted_votes)
    worksheet = sheet.sheet1
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S") # Generate a timestamp in the format 'YYYY-MM-DD HH:MM:SS' to record the exact date and time of the event
    worksheet.append_row([professor, encrypted_vote, timestamp])

def retrieve_votes(encrypted_votes):
    sheet = create_or_open_sheet(encrypted_votes)
    worksheet = sheet.sheet1
    rows = worksheet.get_all_values()

    # Ensure there is data to process if none it should print an error.
    if not rows or len(rows) < 2:
        print("Error: No data or headers found in the sheet.")
        return pd.DataFrame(columns=["Professor", "Encrypted Vote", "Timestamp"])

    df = pd.DataFrame(rows[1:], columns=rows[0])  # Skip header row. Added as I encoutered an error without.
    print("Retrieved DataFrame:")
    print(df.head()) # I used .head here to shorten the list instead of printing the full string.
    return df

def clear_sheet(sheet_name): # Added as once I started running the code the sheet kept duplicaitng the information. This allows me to clear the sheet fully and avoid issues.
    try:
        sheet = gc.open(sheet_name)
        worksheet = sheet.sheet1
        worksheet.clear()
        worksheet.append_row(["Professor", "Encrypted Vote", "Timestamp"])  # Re-add headers
    except SheetNotFound:
        print(f"Sheet {sheet_name} not found.")

# Single voting process to rank candidates based on preferences
def rank_candidates(preferences):
    # Get the set of all candidates from the global name-to-id mapping
    all_candidates = set(name_to_id.keys())

    # List to store the candidates in their final ranked order
    ranked_candidates = []

    # Continue until all candidates are ranked
    while len(ranked_candidates) < len(all_candidates):
        # Initialise a dictionary to count votes for each unranked candidate
        vote_counts = {candidate: 0 for candidate in all_candidates - set(ranked_candidates)}

        # Process each professor's ranked preferences
        for prefs in preferences:
            # For each professor's preference list, find the highest-priority unranked candidate
            for candidate in prefs:
                if candidate not in ranked_candidates:
                    # Increment the vote count for this candidate
                    vote_counts[candidate] += 1
                    break  # Stop after voting for the first valid unranked candidate

        # Determine the candidate with the most votes in this round
        winner = max(vote_counts, key=vote_counts.get)

        # Add the winner to the ranked candidates list
        ranked_candidates.append(winner)

    # Return the final ranking of candidates
    return ranked_candidates


    # Return the fully ranked list of candidates
    return ranked_candidates


# Main program
def main():
    encrypted_votes = "Faculty_Voting"
    clear_sheet(encrypted_votes)  # Clear sheet before adding new votes

    # Add votes to Google Sheet
    for professor, preferences in professor_preferences.items():
        numeric_preferences = encode_preferences(preferences)
        encrypted_preferences = encrypt(numeric_preferences)
        add_vote_to_sheet(encrypted_votes, professor, encrypted_preferences)

    print("Votes encrypted and added to Google Sheet.")

    # Retrieve votes and decrypt
    df = retrieve_votes(encrypted_votes)
    if "Encrypted Vote" not in df.columns:
        print("Error: 'Encrypted Vote' column not found in the data.")
        return
# This ensures the data can be processed numerically and  handles any unexpected or corrupted inputs
    df["Encrypted Vote"] = pd.to_numeric(df["Encrypted Vote"], errors="coerce")
    df["Decrypted Vote"] = df["Encrypted Vote"].apply(decrypt)
    print("Decrypted Votes (before decoding):")
    print(df["Decrypted Vote"])
    df["Decoded Preferences"] = df["Decrypted Vote"].apply(decode_preferences)

    print("Decrypted votes retrieved from Google Sheet.")
    print(df)

    # Aggregate and rank candidates
    decoded_preferences = df["Decoded Preferences"].tolist()
    rankings = rank_candidates(decoded_preferences)

    print("\nFinal Rankings:")
    for i, candidate in enumerate(rankings, 1):
        print(f"{i}. {candidate}")

if __name__ == "__main__":
    main()


Sheet Faculty_Voting not found.
Votes encrypted and added to Google Sheet.
Retrieved DataFrame:
        Professor Encrypted Vote            Timestamp
0    Prof. Ashley           2982  2024-12-06 17:05:32
1     Prof. Baker           4835  2024-12-06 17:05:33
2    Prof. Barnes           8335  2024-12-06 17:05:34
3  Prof. Caroline           7788  2024-12-06 17:05:34
4    Prof. Culver           4835  2024-12-06 17:05:35
Decrypted Votes (before decoding):
0     4123
1     2314
2     3214
3     3124
4     2314
5     3412
6     1423
7     2341
8     2341
9     3412
10    2143
11    2143
12    1243
13    1324
14    2341
15    2134
16    2431
17    2431
18    2143
19    2314
20    4213
Name: Decrypted Vote, dtype: int64
Decrypted votes retrieved from Google Sheet.
           Professor  Encrypted Vote            Timestamp  Decrypted Vote  \
0       Prof. Ashley            2982  2024-12-06 17:05:32            4123   
1        Prof. Baker            4835  2024-12-06 17:05:33            2314   
2  